In [1]:
import requests
from urllib.parse import urljoin

def get_nemo_api_version(base_url: str, host_header: str | None = None, verify_tls: bool = False, timeout: int = 5):
    """
    Attempt to retrieve API/service version from a NeMo Microservices endpoint.
    Tries OpenAPI first, then common health/version endpoints.
    """
    headers = {"Host": host_header} if host_header else {}

    candidates = [
        "openapi.json",
        "docs",  # not version, but useful to see if exists
        "version",
        "v1/version",
        "health",
        "v1/health",
        "v1/healthz",
        "v1/status",
    ]

    results = []

    for path in candidates:
        url = urljoin(base_url.rstrip("/") + "/", path)
        try:
            r = requests.get(url, headers=headers, verify=verify_tls, timeout=timeout)
            ct = (r.headers.get("content-type") or "").lower()

            entry = {"url": url, "status": r.status_code, "content_type": ct, "data": None}

            if r.status_code == 200 and "application/json" in ct:
                data = r.json()
                entry["data"] = data

                # OpenAPI best-case
                if path == "openapi.json":
                    info = data.get("info", {}) if isinstance(data, dict) else {}
                    entry["openapi_title"] = info.get("title")
                    entry["openapi_version"] = info.get("version")
                    entry["openapi_description"] = info.get("description")

            results.append(entry)

        except requests.RequestException as e:
            results.append({"url": url, "error": str(e)})

    # Prefer OpenAPI version if present
    for e in results:
        if e.get("openapi_version"):
            return {
                "base_url": base_url,
                "host_header": host_header,
                "version_source": e["url"],
                "version": e["openapi_version"],
                "title": e.get("openapi_title"),
                "raw": e.get("data"),
                "probes": results,
            }

    # Otherwise return probe results so you can see what responded
    return {
        "base_url": base_url,
        "host_header": host_header,
        "version_source": None,
        "version": None,
        "title": None,
        "raw": None,
        "probes": results,
    }


def print_version_report(report: dict):
    print(f"\n=== {report['base_url']} (Host={report['host_header']}) ===")
    if report["version"]:
        print(f"OpenAPI version: {report['version']}")
        if report["title"]:
            print(f"Title: {report['title']}")
        print(f"Source: {report['version_source']}")
    else:
        print("No OpenAPI version found. Probe results:")
        for p in report["probes"]:
            if "error" in p:
                print(f"  ERR  {p['url']} -> {p['error']}")
            else:
                extra = ""
                if p.get("content_type", "").startswith("application/json") and isinstance(p.get("data"), dict):
                    # show a few keys to help identify response
                    extra = f" keys={list(p['data'].keys())[:8]}"
                print(f"  {p['status']:>3}  {p['url']} ({p.get('content_type','')}){extra}")


# --- Examples ---

# If you can resolve ingress hostnames locally:
for host in ["http://nemo.test", "http://nim.test", "http://data-store.test"]:
    rep = get_nemo_api_version(host, verify_tls=False)
    print_version_report(rep)

# If you must hit the minikube IP, pass the Host header so ingress matches:
minikube_ip = "http://192.168.49.2"
for host_header in ["nemo.test", "nim.test", "data-store.test"]:
    rep = get_nemo_api_version(minikube_ip, host_header=host_header, verify_tls=False)
    print_version_report(rep)



=== http://nemo.test (Host=None) ===
No OpenAPI version found. Probe results:
  404  http://nemo.test/openapi.json (text/html)
  404  http://nemo.test/docs (text/html)
  404  http://nemo.test/version (text/html)
  404  http://nemo.test/v1/version (text/html)
  404  http://nemo.test/health (text/html)
  404  http://nemo.test/v1/health (text/html)
  404  http://nemo.test/v1/healthz (text/html)
  404  http://nemo.test/v1/status (text/html)

=== http://nim.test (Host=None) ===
No OpenAPI version found. Probe results:
  404  http://nim.test/openapi.json (text/html)
  404  http://nim.test/docs (text/html)
  404  http://nim.test/version (text/html)
  404  http://nim.test/v1/version (text/html)
  404  http://nim.test/health (text/html)
  404  http://nim.test/v1/health (text/html)
  404  http://nim.test/v1/healthz (text/html)
  404  http://nim.test/v1/status (text/html)

=== http://data-store.test (Host=None) ===
No OpenAPI version found. Probe results:
  404  http://data-store.test/openapi.js